In [8]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
llm=ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature=0.5)

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [39]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

{response_json}

"""

In [40]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [41]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [42]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [43]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'], template=TEMPLATE2)

In [44]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key='review',verbose=True)

In [45]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], input_variables=['text','number','subject','tone',
                                                                                           'response_json'],
                                        output_variables=['quiz','review'],verbose=True)

In [46]:
file_path=r'D:\ProjectsAI\mcqgen\mcqgen\data.txt'

In [47]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [48]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [49]:
NUMBER=2 
SUBJECT="generative ai"
TONE="complex"

In [50]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[20] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[21] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[22][23] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet's automato

In [51]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1653
Prompt Tokens:1402
Completion Tokens:251
Total Cost:0.002605


In [52]:
response

{'text': 'The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[20] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[21] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[22][23] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet\'s automaton created in the early 1800s.[24]\n\nArtificial Intelligence is an idea that has been captivating

In [56]:
quiz_str=response.get('quiz')
quiz_dict=json.loads(quiz_str)
quiz_dict

{'1': {'mcq': 'What was the name of the computer program created by Harold Cohen to generate paintings in the early 1970s?',
  'options': {'a': 'Turing Machine',
   'b': 'AARON',
   'c': 'Markov Chain',
   'd': 'GPT-4'},
  'correct': 'b'},
 '2': {'mcq': 'Which advancement in 2014 produced the first practical deep neural networks capable of learning generative models for complex data such as images?',
  'options': {'a': 'Variational Autoencoder',
   'b': 'Generative Adversarial Network',
   'c': 'Transformer Network',
   'd': 'Markov Chain'},
  'correct': 'b'}}

In [57]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [59]:
quiz_table_data
df=pd.DataFrame(quiz_table_data)
df

,MCQ,Choices,Correct
0,What was the name of the computer program crea...,a: Turing Machine | b: AARON | c: Markov Chain...,b
1,Which advancement in 2014 produced the first p...,a: Variational Autoencoder | b: Generative Adv...,b


In [ ]:
df.to_csv('Quiz_generative_ai.csv', index=False)